# For this part, we read in authors data and update bibtex key.

In [4]:
import pandas as pd

In [143]:
mergeable = pd.read_csv("merged_pubs.csv")
authors = pd.read_csv("authors.csv")
authors_pubs = pd.read_csv("authors_publications.csv")

In [136]:
need_fix = ~mergeable["bibtexkey"].str.contains('[A-Za-z]')
to_fix = mergeable[need_fix]
rest = mergeable[~need_fix]

In [137]:
to_fix_with_join = to_fix.merge(authors_pubs, how = 'inner', left_on = "id", right_on = 'publication_id')

In [138]:
to_fix_with_authors = to_fix_with_join.merge(authors, how = "inner", left_on = "author_id", right_on = "id")

In [139]:
to_fix_grouped = to_fix_with_authors.groupby("id_x").first()
to_fix_grouped

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,first,east_asian_order,email,institution_y,biography,modified,orcid_id,deceased,ojs_author_id,org_script_name
id_x,,,,,,,,,,,,,,,,,,,,,
1131,MSVO 1,1991-1131,1991,NaN,None,NaN,None,NaN,NaN,NaN,...,Robert K.,0,None,University of California Los Angeles,"<p>Until his retirement in July 2018,&nbsp;<st...",2022-03-01 15:05:26,None,1,NaN,NaN
1138,ZA 072,1982-1138,1982,NaN,None,NaN,None,NaN,NaN,NaN,...,None,0,None,None,None,0000-00-00 00:00:00,None,0,NaN,NaN
1139,ATU 5,1994-1139,1994,NaN,None,NaN,None,NaN,NaN,NaN,...,Robert K.,0,None,University of California Los Angeles,"<p>Until his retirement in July 2018,&nbsp;<st...",2022-03-01 15:05:26,None,1,NaN,NaN
1393,ATU 6,2005-1393,2005,NaN,None,NaN,None,NaN,NaN,NaN,...,Robert K.,0,None,University of California Los Angeles,"<p>Until his retirement in July 2018,&nbsp;<st...",2022-03-01 15:05:26,None,1,NaN,NaN
2611,ATU 7,2001-2611,2001,NaN,None,NaN,None,NaN,NaN,NaN,...,Robert K.,0,None,University of California Los Angeles,"<p>Until his retirement in July 2018,&nbsp;<st...",2022-03-01 15:05:26,None,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240138,JNES 21,1962-240138,1962,NaN,None,NaN,None,NaN,NaN,NaN,...,None,0,None,None,None,0000-00-00 00:00:00,None,0,NaN,NaN
240180,OrNS 37,1968-240180,1968,NaN,None,NaN,None,NaN,NaN,NaN,...,None,0,None,None,None,0000-00-00 00:00:00,None,0,NaN,NaN
240184,ZA 036,1925-240184,1925,NaN,None,NaN,None,NaN,NaN,NaN,...,None,0,None,None,None,0000-00-00 00:00:00,None,0,NaN,NaN


# We ended up filtering out 2 publications that have no author attached as well

In [140]:
not_none = to_fix_grouped["last"].notna() & to_fix_grouped["year"].notna()

In [141]:
to_fix_grouped.loc[not_none, "bibtexkey"] = to_fix_grouped.loc[not_none, "last"] + to_fix_grouped.loc[not_none, "year"]

In [79]:
fixed_merged = pd.concat([to_fix_grouped, rest], axis = 'index').sort_index()

In [145]:
mergeable = mergeable.set_index("id")

In [146]:
mergeable.loc[to_fix_grouped["bibtexkey"].index, "bibtexkey"] = to_fix_grouped["bibtexkey"]

In [149]:
mergeable.to_csv("merged_pubs.csv")